### Goal of this Notebook

Creating specific functions to do API calls to the [RAWG API database](https://rawg.io/apidocs), the most comprehensive videogame database which has a dedicated API.
This notebook consists the following actions : 
2 ad hoc functions, tailromade for the RAWG API, and a for loop function to aggregate all the data from the API calls :
- 1st RAWG_API_call function : calling & appending data to an ad hoc list
- creating_df function : using data, in list format, from the API call, to create an ad hoc dataframe
- final for loop, calling on those two functions, to aggregate all the data into one single comprehensive dataframe 
- exporting dataframe to CSV

**NotaBene List of RAWG Console ID**

Here is the list of all the desired data for each console

**PlayStation**
- **ID#15** : PS2 - games_count': 1737
- **ID#16** : PS3 -  'games_count': 3567
- **ID#18** : PS4 - 'games_count': 4486

**Xbox**
- **ID#80** : Xbox_old - 'games_count': 630
- **ID#14** : Xbox 360 - 'games_count': 2489
- **ID#1** : XboxOne - 'games_count': 3063

**Nintendo**
- **ID#105** : GameCube - 'games_count': 619
- **ID#11** : Wii 'games_count': 2301
- **ID#10** : Wii U - 'games_count': 1272
- **ID#7** : Switch - 'games_count': 3092

Sum of all desired results : final df of 23.3K data

### Importing libraries

In [1]:
import requests
import json
import requests_cache
import pandas as pd
from pandas.io.json import json_normalize
from tqdm import tqdm
import time
from IPython.core.display import clear_output

In [2]:
pd.set_option('display.max_columns', None)

In [3]:
# keeping data from API calls into cache
requests_cache.install_cache()

### Defining functions

**API call function**

In [4]:
def RAWG_API_call(platform_ID):
    """
    API call for the desired console
    cf. list of console #ID - each console has a dedicated RAWG #ID to input
    
    Input: platfrom_ID from the list of RAWG's console ID 
    Output: raw list of the console API results to be Pandas' framed
    """
    #list of game to append API call's results
    list_of_console_game = []

    page=1
    final_page = 420 #dummy pagination
    
    #while loop making API call on desired platfrom up to end of page result
    while page <= final_page:
        print("Requesting page {}/{}".format(page, final_page))
        # clear the output to make things neater
        clear_output(wait = True)

        url = f'https://api.rawg.io/api/games?platforms={platform_ID}&page={page}&page_size=40&ordering=-rating'
        limit= 100

        response = requests.get(url)
        p = {'limit':limit}
        result = response.json()

        if response.status_code !=200:
            print('limit page result reached on page number',page,response.text, '- end of API call')
            break

        list_of_console_game.append(result)

        #if it's not a cached result, time sleep before following call
        if not getattr(response, 'from_cache', False):
            time.sleep(0.30)

        #increment page number
        page += 1
    
    print("Requested page {}/{}".format(page, page))
    
    return list_of_console_game

**Data wrangling for DF**

In [5]:
def creating_df(list_of_console_game, console_name):
    """
    Raw data from RAWG API call transformed into df
    Keeping only limited number of relevant columns : id, name, release date, ratings, genre
    
    Input: list of results from API call + name of relevant console 
    Output: clean df + local CSV file
    """
    
    # Concatenating all console's API calls in one general console dataframe
    frame = []
    for i in range(1,len(list_of_console_game)-1):
        pd_console_temp = pd.DataFrame(list_of_console_game[i]['results'])
        frame.append(pd_console_temp)
    pd_console = pd.concat(frame, sort=False)
    
    #Keeping only specific interesting columns for the study
    pd_console = pd_console[['id', 'slug', 'name', 'released', 'rating', 'rating_top', 'ratings_count', 'metacritic', 'suggestions_count', 'genres']]
    #renaming colmun
    pd_console.rename(columns={'platform':'console', 'genres':'genre'}, inplace=True)

    #Keeping only one genre per game    
    pd_console['genre'] = pd_console['genre'].apply(lambda x: x[0]["name"] if x else x)
    
    #each game may have multiple console in nested dict. Only keeping relevant console name
    pd_console['console'] = f'{console_name}'

    #saving df to csv
    pd_console.to_csv(f'df_{console_name}.csv')
    
    return pd_console

# , print("dataframe created & CSV saved for console :", console_name)

In [6]:
# Creating dict with all console ID from RAWG to call and append
console_dict = {
                15:'PS2',
                16:'PS3',
                18:'PS4',
                80:'Xbox_old',
                14:'Xbox_360',
                1:'XboxOne',
                105:'GameCube',
                11:'Wii',
                10:'Wii_U',
                7:'Switch'}

### API Calls

For loop with all console ID from the console_dcit
For each loop:
- calling 2 above created functions : API call to RAWG + creating dedicated df for each console
- appending result to a global dataframe which encompasses all consoles

In [7]:
#Creating empty dataframe to append data from each API call for each console into one global df with all consoles combined
temp_data = []
df_temp = pd.DataFrame(temp_data)

for console_ID,console_name in console_dict.items():
    print("API call launched for :", console_name) #printing status of current API call
    temp_console = RAWG_API_call(console_ID) #calling function for API call
    print("API call completed for :", console_name) #printing evolution of the API call
    print("-------")
    temp_df = creating_df(temp_console,console_name) # calling function to create temporary df
    temp_data.append(temp_df)                        # appending temporary df data to the empty list
    print("dataframe created & CSV saved for console :", console_name)
    print("-------")

# appending temporary df from the API call to the final df_all_console
df_all_console = df_temp.append(temp_data)
df_all_console

limit page result reached on page number 82 {"detail":"Invalid page."} - end of API call
Requested page 82/82
API call completed for : Switch
-------
dataframe created & CSV saved for console : Switch
-------


,id,slug,name,released,rating,rating_top,ratings_count,metacritic,suggestions_count,genre,console
0,59250,suikoden-v,Suikoden V,2006-02-23,4.38,5,8,76,461,RPG,PS2
1,13925,prince-of-persia-warrior-within,Prince of Persia: Warrior Within,2004-11-30,4.38,5,395,83,641,Adventure,PS2
2,53563,ssx-tricky,SSX Tricky,2001-11-05,4.38,5,26,NaN,303,Racing,PS2
3,23755,dragon-quest-viii-journey-of-the-cursed-king,Dragon Quest VIII: Journey of the Cursed King,2004-11-27,4.38,5,32,84,204,RPG,PS2
4,39707,god-of-war-ii,God of War II,2007-03-13,4.38,5,501,NaN,595,Adventure,PS2
...,...,...,...,...,...,...,...,...,...,...,...
35,60796,double-cross,Double Cross,2019-01-10,0.00,0,4,72,221,Indie,Switch
36,258398,summer-in-mara,Summer in Mara,None,0.00,0,0,NaN,393,Indie,Switch
37,295102,hell-is-other-demons,Hell is Other Demons,2019-04-18,0.00,0,3,NaN,355,Indie,Switch
38,388383,spirit-of-the-north,Spirit of the North,2019-11-01,0.00,0,1,NaN,588,Indie,Switch


In [8]:
df_all_console.to_csv('../database/df_all_console.csv', index=False)